## Imports

In [1]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype

## Functions

In [2]:
import sys
sys.path.append('/content/')

In [3]:
import a_DataHelpers
import b_AlgoFunctions
import c_PredEval
import d_CorrScores

In [5]:
# import sys
# sys.path.append('/content/drive/MyDrive/ColabNotebooks/Research/Functions/')

In [4]:
from a_DataHelpers import getPivotMonths,Time_DataSplit,TestTrain_DataSplit,SingleRatingMatrix
from a_DataHelpers import AllRatingMatrices,All_SingleStepRatMat,psiStep_RatMat,psiAllStep_RatMat
from a_DataHelpers import Find_NewUsersItems,adjustedPSI_DF,adjustedAllDF,get_NEWHoldout,ADJUST_mainDF

from b_AlgoFunctions import integrator,getStartingValues,integratorOnMat,last_psiTrainMat
from b_AlgoFunctions import Updt_RowMatrix,getRow_Mat,row_update, Updt_ColMatrix,getCol_Mat,colunm_update
from b_AlgoFunctions import UsersItems_RatPair,getRowCol_psiupdt,ITEMS_defferredStatus,USERS_defferredStatus
from b_AlgoFunctions import getV_listUpdate, SingleStep_UPDATE,ALLSTEPs_UPDATE

from c_PredEval import TopNPred,TQDMgetALLTopNPred, Hitrate_Eval,getAll_HitRate
from c_PredEval import TQDMgetALLTopNPred_ALLUSERS,getALLTopNPred_ALLUSERS
from c_PredEval import get_ALLRandPred,getAll_RandomHitRate,getMOSTPOP_Pred,getAll_MostPOPHitRate,getAll_MOSTPOP_Pred
from c_PredEval import topN_Index,get_ALLRandPred_2,getAll_RandomHitRate_2,getAll_MostPOPHitRate2

from d_CorrScores import no_copy_csr_matrix,build_rank_weights_matrix,rank_weighted_jaccard_index
from d_CorrScores import Updt_getAll_AvgCorr, updtCorr_4AllRanks

## Apply Functions on *AmazonBeauty*

###Step 1: Data Cleaning

In [ ]:
BeautyDFF = BeautyDFF.drop_duplicates()
BeautyDFF.shape

In [ ]:
sample_df = BeautyDFF.groupby(['userId_int']).agg({'productId_int':'count'}) 
sample_df = sample_df.rename(columns = {'productId_int':'Product_Count'}).reset_index() 
sample_ = sample_df[sample_df['Product_Count'] > 1]         #get list of users that have rated more than 5 products
print(sample_.shape[0])
onetime_userslist =  sample_['userId_int'].to_list()
#print(len(onetime_userslist))
#onetime_userslist[:10]

In [ ]:
# df_SAMPLE = BeautyDFF.groupby(['productId_int']).agg({'userId_int':'count'}) 
# df_SAMPLE = df_SAMPLE.rename(columns = {'userId_int':'User_Count'}).reset_index() 
# DF_Items = df_SAMPLE[df_SAMPLE['User_Count'] > 1]         #get list of users that have rated more than 5 products
# print(DF_Items.shape[0])
# onetime_itemslist =  DF_Items['productId_int'].to_list()
# print(len(onetime_itemslist))
# #onetime_itemslist[:10]

In [ ]:
DFF_ = BeautyDFF.loc[ BeautyDFF['userId_int'].isin(onetime_userslist) ]#& BeautyDFF['productId_int'].isin(onetime_itemslist)]
print(DFF_['productId_int'].nunique())
print(DFF_['userId_int'].nunique())

In [ ]:
#AMZB_1 = BeautyDFF.loc[BeautyDFF['userId_int'].isin(onetime_userslist) & BeautyDFF['productId_int'].isin(onetime_userslist)]
AMZB_1 = DFF_[['productId_int','userId_int','rating','timestamp']]
AMZB_1

In [ ]:
#AMZB_df = AMZB_11.rename(columns={'userId':'userId_int','productId':'productId_int'})
#AMZB_1['timestamp'] = pd.to_datetime(AMZB_1['timestamp']) 
AMZB_df = AMZB_1.copy()
print(AMZB_df.dtypes)
print()
print("nUnique users: ",AMZB_df['userId_int'].nunique())
print("nUnique items: ",AMZB_df['productId_int'].nunique())
AMZB_df = AMZB_df.sort_values(['timestamp'])
AMZB_df

In [ ]:
unique_USER_id = pd.unique(AMZB_df.userId_int)
unique_PRODUCT_id = pd.unique(AMZB_df.productId_int)

zipped_USER_id_pair = zip(unique_USER_id, range(0, len(unique_USER_id)+1, 1))
zipped_PRODUCT_id_pair = zip(unique_PRODUCT_id, range(0, len(unique_PRODUCT_id)+1, 1))

USER_id_dict = OrderedDict(zipped_USER_id_pair)
PRODUCT_id_dict = OrderedDict(zipped_PRODUCT_id_pair)

AMZB_df['userId'] = AMZB_df.userId_int.map(USER_id_dict)
AMZB_df['productId'] = AMZB_df.productId_int.map(PRODUCT_id_dict)
print(AMZB_df['userId'].max())
print(AMZB_df['productId'].max())
print()
print(AMZB_df['userId'].nunique())
print(AMZB_df['productId'].nunique())
AMZB_df

In [ ]:
AMZB_DF = AMZB_df[['userId','productId','rating','timestamp']]
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())
AMZB_DF

###Step 2: Import Clean Data 

In [5]:
AMZB_DF = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Research/Test_Data/usedAMZB.csv')
AMZB_DF['timestamp'] = pd.to_datetime(AMZB_DF['timestamp']) 
print(AMZB_DF.shape)   #productId_int	 userId_int
print(AMZB_DF.dtypes)
AMZB_DF

(51869, 4)
userId                int64
productId             int64
rating                int64
timestamp    datetime64[ns]
dtype: object


,userId,productId,rating,timestamp
0,0,0,1,2001-06-08
1,1,0,1,2002-06-12
2,1,1,1,2002-06-12
3,2,1,1,2002-06-17
4,3,1,1,2002-07-04
...,...,...,...,...
51864,21470,6687,1,2018-09-21
51865,16839,5635,1,2018-09-21
51866,15319,8618,1,2018-09-25
51867,12925,5977,1,2018-09-25


In [6]:
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())

(51869, 4)
For users: 
22799
22798

For items: 
8620
8619

Timestamp
2001-06-08 00:00:00
2018-09-26 00:00:00


###Step 1: Data Splitting

In [7]:
pivotMonths_list = getPivotMonths(AMZB_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-08-26 00:00:00'),
 Timestamp('2018-07-26 00:00:00'),
 Timestamp('2018-06-26 00:00:00'),
 Timestamp('2018-05-26 00:00:00'),
 Timestamp('2018-04-26 00:00:00'),
 Timestamp('2018-03-26 00:00:00'),
 Timestamp('2018-02-26 00:00:00'),
 Timestamp('2018-01-26 00:00:00'),
 Timestamp('2017-12-26 00:00:00'),
 Timestamp('2017-11-26 00:00:00'),
 Timestamp('2017-10-26 00:00:00'),
 Timestamp('2017-09-26 00:00:00'),
 Timestamp('2017-08-26 00:00:00'),
 Timestamp('2017-07-26 00:00:00'),
 Timestamp('2017-06-26 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-04-26 00:00:00'),
 Timestamp('2017-03-26 00:00:00')]

In [8]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZB_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-03-25 00:00:00
Num ΔA_train : 10


In [9]:
AllDF_list, PSITest_list, HOLDOUT_list, UserItemDF_list = TestTrain_DataSplit(AMZB_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [10]:
print(AllDF_list[0].shape)
print(UserItemDF_list[0].shape)
print(HOLDOUT_list[0].shape)

(50377, 4)
(50119, 4)
(258, 4)


### Step 2: Find new users and items

In [11]:
Tstps = AMZB_DF['timestamp']
tr_ts = ΔA_train[-1]['timestamp'].max()
print("ΔA_train_last max date: ",tr_ts)
AllDF_start = AMZB_DF.loc[Tstps <= tr_ts] 
print(AllDF_start['timestamp'].min())
print(AllDF_start['timestamp'].max())
print()
print(AllDF_list[0]['timestamp'].min())
print(AllDF_list[0]['timestamp'].max())

ΔA_train_last max date:  2018-01-25 00:00:00
2001-06-08 00:00:00
2018-01-25 00:00:00

2001-06-08 00:00:00
2018-02-25 00:00:00


In [12]:
New_usersList,New_itemsList = Find_NewUsersItems(AllDF_start,AllDF_list,'userId','productId',N_steps=8)
New_usersList[0]   #

array([22340, 22341, 22342, 22343, 22344, 22345, 22346, 22347, 22348,
       22349, 22350, 22351, 22352, 22353, 22354, 22355, 22356, 22357,
       22358, 22359, 22360, 22361, 22362, 22363, 22364, 22365, 22366,
       22367, 22368, 22369, 22370, 22371, 22372, 22373, 22374, 22375,
       22376, 22377, 22378, 22379, 22380, 22381, 22382, 22383, 22384,
       22385, 22386, 22387, 22388, 22389, 22390, 22391, 22392, 22393,
       22394, 22395, 22396, 22397, 22398, 22399, 22400, 22401, 22402,
       22403, 22404, 22405, 22406, 22407, 22408, 22409, 22410, 22411,
       22412, 22413, 22414, 22415, 22416, 22417, 22418, 22419, 22420,
       22421, 22422, 22423, 22424, 22425, 22426, 22427, 22428, 22429,
       22430, 22431, 22432, 22433, 22434, 22435, 22436, 22437, 22438,
       22439, 22440, 22441, 22442, 22443, 22444, 22445, 22446, 22447,
       22448, 22449, 22450])

In [13]:
New_itemsList[0]   

array([8328, 8329, 8330, 8331, 8332, 8333, 8334, 8335, 8336, 8337, 8338,
       8339, 8340, 8341, 8342, 8343, 8344, 8345, 8346, 8347, 8348, 8349,
       8350, 8351, 8352, 8353, 8354, 8355, 8356, 8357, 8358, 8359, 8360,
       8361, 8362, 8363, 8364, 8365, 8366, 8367, 8368, 8369, 8370, 8371,
       8372, 8373, 8374, 8375, 8376, 8377, 8378, 8379, 8380, 8381, 8382,
       8383, 8384, 8385, 8386, 8387, 8388, 8389, 8390, 8391, 8392, 8393,
       8394, 8395, 8396, 8397, 8398])

In [14]:
for i,j in zip(New_usersList,New_itemsList):
    print(i.size)
    print(j.size)
    print()

111
71

97
56

119
44

78
36

24
26

17
28

12
20

1
11



###Get Updt RatMat

In [15]:
rows_i = AllDF_start['userId'].nunique()
cols_i = AllDF_start['productId'].nunique()
print(A0_df.shape)
A0_RatMat_updt = SingleRatingMatrix(A0_df,'userId', 'productId',rows_i,cols_i)
A0_RatMat_updt

(45139, 4)


<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
	with 44740 stored elements in Compressed Sparse Row format>

In [16]:
PSI_TrainMat_updt = AllRatingMatrices(ΔA_train,'userId','productId',rows_i ,cols_i)
PSI_TrainMat_updt

[<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 553 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 518 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 432 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 397 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 1017 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 482 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 357 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse mat

### Allsteps Update

In [ ]:
# rows_i = AllDF_start['userId'].nunique()
# cols_i = AllDF_start['productId'].nunique()
# print(AllDF_start.shape)
# AllDF_startRatMat = SingleRatingMatrix(AllDF_start,'userId', 'productId',rows_i,cols_i)

# U_start,S_start,V_start = svds(AllDF_startRatMat,k=50)
# V_start = V_start.T
# print()
# print("U_start shape: ",U_start.shape)
# print("S_start shape: ",S_start.shape)
# print("V_start shape: ",V_start.shape)

In [17]:
U_start,S_start,V_start = last_psiTrainMat(A0_RatMat_updt,PSI_TrainMat_updt,k=50)
print()
print("U_start shape: ",U_start.shape)
print("S_start shape: ",S_start.shape)
print("V_start shape: ",V_start.shape)

100%|██████████| 10/10 [00:01<00:00,  9.67it/s]


U_start shape:  (22340, 50)
S_start shape:  (50, 50)
V_start shape:  (8328, 50)


In [18]:
U_list,S_list,V_list = [],[],[]
U_list.append(U_start)
S_list.append(S_start)
V_list.append(V_start)
print(len(U_list),len(V_list))

1 1


In [19]:
DItems_, DUsers_,In_DomainUSERS,In_DomainITEMS,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,U_list,S_list,V_list = ALLSTEPs_UPDATE(AllDF_start,
                                            UserItemDF_list,New_itemsList,New_usersList,U_list,S_list,V_list,'userId','productId',Nsteps=8,k=50,Forced_Orth=False)

100%|██████████| 7/7 [00:24<00:00,  3.50s/it]


#### Output Check

In [20]:
print(len(In_DomainITEMS[8327+1:]))
print(In_DomainITEMS[8327+1:])
print()
print(len(In_DomainUSERS[22339+1:]))
print(In_DomainUSERS[22339+1:])

236
[8381, 8448, 8449, 8329, 8330, 8331, 8332, 8333, 8336, 8343, 8344, 8345, 8346, 8348, 8349, 8352, 8355, 8361, 8362, 8363, 8364, 8365, 8368, 8369, 8370, 8371, 8372, 8373, 8374, 8375, 8378, 8379, 8380, 8385, 8386, 8388, 8389, 8391, 8398, 8435, 8328, 8334, 8340, 8353, 8357, 8367, 8376, 8377, 8383, 8384, 8390, 8392, 8395, 8399, 8400, 8401, 8404, 8405, 8406, 8407, 8408, 8410, 8413, 8414, 8415, 8416, 8417, 8418, 8419, 8422, 8423, 8424, 8427, 8428, 8431, 8432, 8433, 8434, 8437, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8450, 8451, 8452, 8453, 8454, 8468, 8409, 8411, 8412, 8420, 8436, 8360, 8506, 8517, 8455, 8456, 8457, 8458, 8459, 8461, 8464, 8465, 8467, 8469, 8470, 8471, 8472, 8474, 8475, 8476, 8478, 8479, 8481, 8482, 8483, 8484, 8487, 8489, 8490, 8495, 8351, 8507, 8463, 8421, 8350, 8499, 8500, 8501, 8504, 8505, 8508, 8509, 8510, 8511, 8512, 8513, 8514, 8515, 8516, 8518, 8520, 8521, 8522, 8523, 8524, 8530, 8531, 8532, 8354, 8519, 8525, 8526, 8529, 8460, 8498, 8356, 8535,

In [21]:
#deferred items and users
print(len(DItems_))
print(DItems_)
print()
print(len(DUsers_))
print(DUsers_)

56
[8609, 8610, 8612, 8614, 8615, 8616, 8617, 8618, 8619, 8566, 8567, 8570, 8572, 8573, 8585, 8586, 8546, 8547, 8502, 8503, 8527, 8528, 8533, 8534, 8466, 8473, 8477, 8480, 8485, 8486, 8488, 8491, 8492, 8493, 8494, 8496, 8497, 8402, 8403, 8425, 8426, 8429, 8430, 8335, 8338, 8339, 8341, 8342, 8347, 8358, 8359, 8382, 8393, 8394, 8396, 8397]

23
[22771, 22776, 22785, 22751, 22677, 22734, 22742, 22613, 22637, 22646, 22651, 22653, 22659, 22453, 22487, 22497, 22355, 22362, 22365, 22372, 22388, 22443, 22446]


In [22]:
for x in list(itemID_dict)[8355:]:
    print ("Id: {}, UpdtId: {} ".format(x,  itemID_dict[x]))   ##id ==key || updtedid == values  ,itemID_dict

Id: 8373, UpdtId: 8355 
Id: 8374, UpdtId: 8356 
Id: 8375, UpdtId: 8357 
Id: 8378, UpdtId: 8358 
Id: 8379, UpdtId: 8359 
Id: 8380, UpdtId: 8360 
Id: 8385, UpdtId: 8361 
Id: 8386, UpdtId: 8362 
Id: 8388, UpdtId: 8363 
Id: 8389, UpdtId: 8364 
Id: 8391, UpdtId: 8365 
Id: 8398, UpdtId: 8366 
Id: 8435, UpdtId: 8367 
Id: 8328, UpdtId: 8368 
Id: 8334, UpdtId: 8369 
Id: 8340, UpdtId: 8370 
Id: 8353, UpdtId: 8371 
Id: 8357, UpdtId: 8372 
Id: 8367, UpdtId: 8373 
Id: 8376, UpdtId: 8374 
Id: 8377, UpdtId: 8375 
Id: 8383, UpdtId: 8376 
Id: 8384, UpdtId: 8377 
Id: 8390, UpdtId: 8378 
Id: 8392, UpdtId: 8379 
Id: 8395, UpdtId: 8380 
Id: 8399, UpdtId: 8381 
Id: 8400, UpdtId: 8382 
Id: 8401, UpdtId: 8383 
Id: 8404, UpdtId: 8384 
Id: 8405, UpdtId: 8385 
Id: 8406, UpdtId: 8386 
Id: 8407, UpdtId: 8387 
Id: 8408, UpdtId: 8388 
Id: 8410, UpdtId: 8389 
Id: 8413, UpdtId: 8390 
Id: 8414, UpdtId: 8391 
Id: 8415, UpdtId: 8392 
Id: 8416, UpdtId: 8393 
Id: 8417, UpdtId: 8394 
Id: 8418, UpdtId: 8395 
Id: 8419, UpdtId

In [23]:
for x in list(userID_dict)[22339:]:
    print ("Id: {}, UpdtId: {} ".format(x,  userID_dict[x]))   ##id ==key || updtedid == values 

Id: 22339, UpdtId: 22339 
Id: 22340, UpdtId: 22340 
Id: 22341, UpdtId: 22341 
Id: 22342, UpdtId: 22342 
Id: 22345, UpdtId: 22343 
Id: 22346, UpdtId: 22344 
Id: 22348, UpdtId: 22345 
Id: 22349, UpdtId: 22346 
Id: 22350, UpdtId: 22347 
Id: 22351, UpdtId: 22348 
Id: 22352, UpdtId: 22349 
Id: 22353, UpdtId: 22350 
Id: 22356, UpdtId: 22351 
Id: 22357, UpdtId: 22352 
Id: 22358, UpdtId: 22353 
Id: 22361, UpdtId: 22354 
Id: 22364, UpdtId: 22355 
Id: 22366, UpdtId: 22356 
Id: 22371, UpdtId: 22357 
Id: 22373, UpdtId: 22358 
Id: 22375, UpdtId: 22359 
Id: 22376, UpdtId: 22360 
Id: 22378, UpdtId: 22361 
Id: 22382, UpdtId: 22362 
Id: 22383, UpdtId: 22363 
Id: 22385, UpdtId: 22364 
Id: 22387, UpdtId: 22365 
Id: 22390, UpdtId: 22366 
Id: 22391, UpdtId: 22367 
Id: 22392, UpdtId: 22368 
Id: 22393, UpdtId: 22369 
Id: 22394, UpdtId: 22370 
Id: 22395, UpdtId: 22371 
Id: 22396, UpdtId: 22372 
Id: 22397, UpdtId: 22373 
Id: 22398, UpdtId: 22374 
Id: 22400, UpdtId: 22375 
Id: 22402, UpdtId: 22376 
Id: 22403, U

In [24]:
print(len(U_list),len(V_list))
print(U_list[0].shape,V_list[0].shape)
print()
for u,v in zip(U_list[1:],V_list[1:]):
    print(u.shape,v.shape)   

9 9
(22340, 50) (8328, 50)

(22407, 50) (8329, 50)
(22511, 50) (8381, 50)
(22634, 50) (8429, 50)
(22706, 50) (8462, 50)
(22738, 50) (8493, 50)
(22761, 50) (8522, 50)
(22772, 50) (8542, 50)
(22776, 50) (8564, 50)


In [25]:
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [26]:
print(len(In_DomainUSERS),len(In_DomainITEMS))

22776 8564


Data Adjustments

#### Data Adjustment

In [27]:
newHOLDOUT_LIST = get_NEWHoldout(HOLDOUT_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,'userId','productId',n=8)
print(len(newHOLDOUT_LIST))
print()
for old,new in zip(HOLDOUT_list,newHOLDOUT_LIST):
    print(old.shape[0],new.shape[0])

100%|██████████| 8/8 [00:00<00:00, 167.02it/s]

8

258 178
239 173
253 207
207 165
148 115
97 72
66 44
30 21


In [28]:
newUserItem_list = adjustedAllDF(UserItemDF_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,'userId','productId',n=8)
print()
for df in newUserItem_list:
    print(df['Updated_UserID'].nunique(),df['Updated_ItemID'].nunique())


22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [29]:
new_PSIDFlist = adjustedPSI_DF(PSITest_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,'userId','productId',n=8)
for old,new in zip(PSITest_list,new_PSIDFlist):
    print(old.shape[0],new.shape[0])

110 89
104 96
178 171
114 106
25 21
18 14
10 9
3 3


#### Get UserItem MAT

In [30]:
UserItem_Mat = All_SingleStepRatMat(newUserItem_list,'Updated_UserID','Updated_ItemID')
print(len(UserItem_Mat))
UserItem_Mat

8


[<22407x8329 sparse matrix of type '<class 'numpy.float64'>'
 	with 49611 stored elements in Compressed Sparse Row format>,
 <22511x8381 sparse matrix of type '<class 'numpy.float64'>'
 	with 49953 stored elements in Compressed Sparse Row format>,
 <22634x8429 sparse matrix of type '<class 'numpy.float64'>'
 	with 50358 stored elements in Compressed Sparse Row format>,
 <22706x8462 sparse matrix of type '<class 'numpy.float64'>'
 	with 50707 stored elements in Compressed Sparse Row format>,
 <22738x8493 sparse matrix of type '<class 'numpy.float64'>'
 	with 50940 stored elements in Compressed Sparse Row format>,
 <22761x8522 sparse matrix of type '<class 'numpy.float64'>'
 	with 51108 stored elements in Compressed Sparse Row format>,
 <22772x8542 sparse matrix of type '<class 'numpy.float64'>'
 	with 51211 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 51280 stored elements in Compressed Sparse Row format>]

In [31]:
print(len(U_list),len(V_list))
for u,v in zip(U_list,V_list):
    print(u.shape[0],v.shape[0])   

9 9
22340 8328
22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [32]:
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())
print()
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22340 8328

22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


#### Prediction

In [41]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [33]:
Vpsi_listUpdt = V_list[1:]
for v in Vpsi_listUpdt:
    print(v.shape)

(8329, 50)
(8381, 50)
(8429, 50)
(8462, 50)
(8493, 50)
(8522, 50)
(8542, 50)
(8564, 50)


In [34]:
V_1 = Vpsi_listUpdt[0]
V_1.shape

(8329, 50)

In [35]:
Top_NPred = TopNPred(UserItem_Mat[0],newHOLDOUT_LIST[0],V_1,'Updated_UserID',10)
Top_NPred

array([[ 519,  381,   63, ...,  922,   83,   85],
       [  63,  520,   43, ...,  137,   36,  134],
       [ 519,   80,   41, ...,  667, 5977, 1725],
       ...,
       [1566,  538, 2031, ..., 1375, 2513, 3030],
       [ 311,   85,  191, ...,  613, 1725, 1049],
       [  67,   83,  416, ..., 1392,  393,  389]])

In [36]:
Hitrate_Eval(newHOLDOUT_LIST[0],Top_NPred,'Updated_UserID','Updated_ItemID')

Number of hits:  53
Total Num of users:  178
Recommendation HitRate:  0.29775280898876405


0.29775280898876405

In [37]:
AllUpdt_pred =  TQDMgetALLTopNPred(UserItem_Mat,newHOLDOUT_LIST,Vpsi_listUpdt,'Updated_UserID',N = 10)
print(len(AllUpdt_pred))
AllUpdt_pred[:2]

8it [00:00, 38.66it/s]

8


[array([[ 519,  381,   63, ...,  922,   83,   85],
        [  63,  520,   43, ...,  137,   36,  134],
        [ 519,   80,   41, ...,  667, 5977, 1725],
        ...,
        [1566,  538, 2031, ..., 1375, 2513, 3030],
        [ 311,   85,  191, ...,  613, 1725, 1049],
        [  67,   83,  416, ..., 1392,  393,  389]]),
 array([[1260,  194,   85, ...,  922, 1512,  558],
        [1260,  194,   85, ...,  922, 1512,  558],
        [ 389,  133,  598, ..., 1034,   82, 1310],
        ...,
        [  68,   69, 1512, ...,   85, 2199,   83],
        [  74,  231,  137, ...,   69, 1512, 2736],
        [ 995,  717,  692, ..., 4017, 3293,  916]])]

In [38]:
AllSteps_HitR_updt, LowerBand_updt, Avg_HitR_updt, UpperBand_updt   = getAll_HitRate(newHOLDOUT_LIST,AllUpdt_pred,'Updated_UserID','Updated_ItemID')

Number of hits:  53
Total Num of users:  178
Recommendation HitRate:  0.29775280898876405
Number of hits:  53
Total Num of users:  173
Recommendation HitRate:  0.3063583815028902
Number of hits:  81
Total Num of users:  207
Recommendation HitRate:  0.391304347826087
Number of hits:  45
Total Num of users:  165
Recommendation HitRate:  0.2727272727272727
Number of hits:  11
Total Num of users:  115
Recommendation HitRate:  0.09565217391304348
Number of hits:  4
Total Num of users:  72
Recommendation HitRate:  0.05555555555555555
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.1774188175642016


### Regular PSI 

#### Get Rating Matrices

In [39]:
newAMZB_DF = ADJUST_mainDF(AMZB_DF,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,'userId','productId')
print(newAMZB_DF.shape)
print(newAMZB_DF['userId'].max(),newAMZB_DF['productId'].max())
print(newAMZB_DF['userId'].nunique(),newAMZB_DF['productId'].nunique())
print()
print(newAMZB_DF['Updated_UserID'].max(),newAMZB_DF['Updated_ItemID'].max())
print(newAMZB_DF['Updated_UserID'].nunique(),newAMZB_DF['Updated_ItemID'].nunique())
newAMZB_DF.head()

(51813, 6)
22798 8613
22776 8564

22775 8563
22776 8564


,userId,productId,rating,timestamp,Updated_UserID,Updated_ItemID
0,0,0,1,2001-06-08,0,0
1,1,0,1,2002-06-12,1,0
2,1,1,1,2002-06-12,1,1
3,2,1,1,2002-06-17,2,1
4,3,1,1,2002-07-04,3,1


In [40]:
rows_i = newAMZB_DF['Updated_UserID'].nunique()
cols_i = newAMZB_DF['Updated_ItemID'].nunique()
print(A0_df.shape)
A0_Rating_matrix = SingleRatingMatrix(A0_df,'userId', 'productId',rows_i,cols_i)  ##changes start within steps...
A0_Rating_matrix

(45139, 4)


<22776x8564 sparse matrix of type '<class 'numpy.float64'>'
	with 44740 stored elements in Compressed Sparse Row format>

In [41]:
for df in new_PSIDFlist:
    print(df['Updated_UserID'].max(),df['Updated_ItemID'].max())

22406 8328
22510 8367
22612 8422
22702 8458
22721 7967
22749 8518
22769 8522
22772 8543


In [42]:
print("Length of PSI_Train :",len(ΔA_train))
print("Length of PSI_Test  :",len(new_PSIDFlist))

Length of PSI_Train : 10
Length of PSI_Test  : 8


In [43]:
PSI_train_matrix = AllRatingMatrices(ΔA_train,'userId','productId',rows_i ,cols_i)
PSI_train_matrix   

[<22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 553 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 518 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 432 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 397 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 1017 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 482 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 357 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse mat

In [44]:
PSI_test_matrix = AllRatingMatrices(new_PSIDFlist,'Updated_UserID','Updated_ItemID',rows_i ,cols_i)                                                                      
print(len(PSI_test_matrix))
PSI_test_matrix

8


[<22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 88 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 96 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 171 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 106 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 21 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 14 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 9 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 3 stored elements in Compressed Sparse Row format>]

In [46]:
UserItemMAT_list = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
UserItemMAT_list  

[<22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 49611 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 49953 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 50358 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 50707 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 50940 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 51108 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 51211 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 51280 stored elements in Compressed Sparse Row format>]

In [47]:
print(AllDF_list[-1]['userId'].nunique(),AllDF_list[-1]['productId'].nunique())
print(AMZB_DF['userId'].nunique(),AMZB_DF['productId'].nunique())

22799 8620
22799 8620


#### PSI Prediction

In [48]:
                                                                                   #integratorOnMat(A0,ΔA_train_matrix,ΔA_test_matrix,k):
Vpsi_listReg =  integratorOnMat(A0_Rating_matrix,PSI_train_matrix,PSI_test_matrix,k=50)  #PSI == projector splitting integrator
print(len(Vpsi_listReg))  

8


In [49]:
Top10Pred_psi = TopNPred(UserItemMAT_list[0], newHOLDOUT_LIST[0],Vpsi_listReg[0],'Updated_UserID',N=10)
print(Top10Pred_psi.shape)
Top10Pred_psi

(178, 10)


array([[ 519,  381,   63, ...,  922,   83,   85],
       [  63,  520,   43, ...,  137,   36,  134],
       [ 519,   80,   41, ...,  667, 5977, 1725],
       ...,
       [1566,  538, 2031, ..., 1375, 2513, 3030],
       [ 311,   85,  191, ...,  613, 1725, 1049],
       [  67,   83,  416, ..., 1392,  393,  389]])

In [50]:
Hitrate_Eval(newHOLDOUT_LIST[0],Top10Pred_psi,'Updated_UserID','Updated_ItemID')

Number of hits:  52
Total Num of users:  178
Recommendation HitRate:  0.29213483146067415


0.29213483146067415

In [51]:
All_PSI_PRED =  TQDMgetALLTopNPred(UserItemMAT_list,newHOLDOUT_LIST,Vpsi_listReg,'Updated_UserID',N = 10)
print(len(All_PSI_PRED))
All_PSI_PRED[:2]

8it [00:00, 36.28it/s]

8


[array([[ 519,  381,   63, ...,  922,   83,   85],
        [  63,  520,   43, ...,  137,   36,  134],
        [ 519,   80,   41, ...,  667, 5977, 1725],
        ...,
        [1566,  538, 2031, ..., 1375, 2513, 3030],
        [ 311,   85,  191, ...,  613, 1725, 1049],
        [  67,   83,  416, ..., 1392,  393,  389]]),
 array([[ 815,  239,  717, ...,  676, 3985,  519],
        [ 815,  239,  717, ...,  676, 3985,  519],
        [ 389,  133,  598, ..., 1034,   82, 1310],
        ...,
        [  68,   69, 1512, ...,   85, 2199,   83],
        [  74,  231,  137, ...,   69, 1512,  105],
        [ 995,  717,  692, ..., 4017, 3293,  916]])]

In [52]:
AllSteps_Hitrate_psi, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHOLDOUT_LIST,All_PSI_PRED,'Updated_UserID','Updated_ItemID')

Number of hits:  52
Total Num of users:  178
Recommendation HitRate:  0.29213483146067415
Number of hits:  52
Total Num of users:  173
Recommendation HitRate:  0.30057803468208094
Number of hits:  79
Total Num of users:  207
Recommendation HitRate:  0.38164251207729466
Number of hits:  45
Total Num of users:  165
Recommendation HitRate:  0.2727272727272727
Number of hits:  11
Total Num of users:  115
Recommendation HitRate:  0.09565217391304348
Number of hits:  4
Total Num of users:  72
Recommendation HitRate:  0.05555555555555555
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.17478629755199018


In [53]:
for df in newUserItem_list:
    print(df['Updated_UserID'].nunique(),df['Updated_ItemID'].nunique())

22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


### CORRELATION Report


In [54]:
UserItem_Mat[0].shape
UserItemMAT_list[0].shape

(22776, 8564)

#### getAll User Pred

In [55]:
AllUsers_updtPRED =  TQDMgetALLTopNPred_ALLUSERS(UserItem_Mat,Vpsi_listUpdt, N=10) 
print(len(AllUsers_updtPRED))
AllUsers_updtPRED[:2]

8it [00:27,  3.38s/it]

8


[array([[ 381,  297,   63, ...,  191, 1202, 1512],
        [ 297,   60,   83, ...,  137,   63,  381],
        [ 297,   60, 2255, ...,   41,   83,  215],
        ...,
        [1566,  538, 2031, ..., 1375, 2513, 3030],
        [ 311,   85,  191, ...,  613, 1725, 1049],
        [  67,   83,  416, ..., 1392,  393,  389]]),
 array([[ 381,  297,   63, ...,  191, 1202, 1512],
        [ 297,   60,   83, ...,  137,   63,  381],
        [ 297,   60, 2255, ...,   41,   83,  215],
        ...,
        [ 558,  676,  815, ...,   25,   24,  319],
        [4017,  717, 3293, ..., 1203,  865, 1512],
        [ 519,  381,   60, ...,  520, 1725,  694]])]

In [66]:
AllUsers_updtPRED[2].shape

(22634, 10)

####  SingleRank Corr 

In [56]:
AllSteps = list(range(1,8))
print(len(AllSteps))
print(len(AllUsers_updtPRED))
AllSteps

7
8


[1, 2, 3, 4, 5, 6, 7]

In [57]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(AllUsers_updtPRED,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_

array([array([1.        , 1.        , 1.        , ..., 0.18551237, 0.16007859,
       0.01736733]),
       array([1.        , 1.        , 1.        , ..., 0.49096051, 0.62918   ,
       0.        ]),
       array([1.        , 1.        , 1.        , ..., 0.37346483, 0.91291953,
       0.        ]),
       array([1.        , 1.        , 1.        , ..., 0.42793577, 0.49096051,
       0.49096051]),
       array([1.        , 1.        , 1.        , ..., 0.26246472, 0.67502553,
       1.        ]),
       array([1.        , 1.        , 1.        , ..., 0.02928462, 0.11059284,
       0.7956453 ]),
       array([1.        , 1.        , 1.        , ..., 0.93397092, 1.        ,
       0.96277091])], dtype=object)

In [58]:
UpdtPSICorr_.shape
for df in UpdtPSICorr_:
    print(df.mean())  

0.9709379225751309
0.9805529399419604
0.9853496053939611
0.9906672957847809
0.9938861878282262
0.9953173560574045
0.9948768526616134


#### RegPSI Corr_

In [62]:
AllUsersRegPIS_PRED =  TQDMgetALLTopNPred_ALLUSERS(UserItemMAT_list,Vpsi_listReg, N=10) 
print(len(AllUsersRegPIS_PRED))
AllUsersRegPIS_PRED[:2]

8it [00:29,  3.65s/it]

8


[array([[ 381,  297,   63, ...,  191, 1202, 1512],
        [ 297,   60,   83, ...,  137,   63,  381],
        [ 297,   60, 2255, ...,   41,   83,  215],
        ...,
        [2851, 2849, 2844, ..., 2855, 2856, 8563],
        [2851, 2849, 2844, ..., 2855, 2856, 8563],
        [2851, 2849, 2844, ..., 2855, 2856, 8563]]),
 array([[ 381,  297,   63, ...,  191, 1202, 1512],
        [ 297,   60,   83, ...,  137,   63,  381],
        [ 297,   60, 2255, ...,   41,   83,  215],
        ...,
        [2851, 2849, 2844, ..., 2855, 2856, 8563],
        [2851, 2849, 2844, ..., 2855, 2856, 8563],
        [2851, 2849, 2844, ..., 2855, 2856, 8563]])]

In [63]:
AllUsersRegPIS_PRED[2].shape

(22776, 10)

####  RegPSI SingleRankCorr 

In [64]:
AllSteps = list(range(1,8))
print(len(AllSteps))
print(len(AllUsers_updtPRED))
AllSteps  

7
8


[1, 2, 3, 4, 5, 6, 7]

In [65]:
RegPSICorr_ = Updt_getAll_AvgCorr(AllUsersRegPIS_PRED,newUserItem_list,AllSteps,'Updated_UserID')  #,
RegPSICorr_

array([array([1.        , 1.        , 1.        , ..., 0.18551237, 0.16007859,
       0.01736733]),
       array([1.        , 1.        , 1.        , ..., 0.49096051, 0.62918   ,
       0.        ]),
       array([1.        , 1.        , 1.        , ..., 0.35022409, 0.94467132,
       0.        ]),
       array([1.        , 1.        , 1.        , ..., 0.43278657, 0.49096051,
       0.49096051]),
       array([1.        , 1.        , 1.        , ..., 0.26246472, 0.68650748,
       1.        ]),
       array([1.        , 1.        , 1.        , ..., 0.02928462, 0.11059284,
       0.7956453 ]),
       array([1.        , 1.        , 1.        , ..., 0.93397092, 1.        ,
       0.95109701])], dtype=object)

In [66]:
RegPSICorr_.shape
for df in RegPSICorr_:
    print(df.mean())  

0.9787586445172054
0.9813125601753522
0.9886840006787605
0.9924237348462716
0.9957534362484753
0.996405052481314
0.9975378108378178


###  RandRec:Test

In [67]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [70]:
All_RandPred = get_ALLRandPred(UserItem_Mat,newHOLDOUT_LIST,'Updated_UserID',N=10)
All_RandPred[:2]  

8it [00:00, 37.91it/s]


[array([[1472,  293, 3912, ..., 2614, 7386,  889],
        [5036, 7804, 1724, ..., 5280, 4684, 6452],
        [1691, 5823,  839, ..., 1077,  832, 7213],
        ...,
        [5940, 5588, 4274, ..., 8236, 8134, 8160],
        [1382, 2562, 6012, ..., 7315, 1594, 8244],
        [7109, 5976, 4482, ..., 1479,  774,  825]]),
 array([[4675, 3040, 4012, ..., 6094, 4812,  565],
        [1002, 3876, 3355, ...,  972, 5386,  272],
        [7768, 4253,  570, ..., 6282, 4458, 3884],
        ...,
        [2199, 6073,  398, ..., 2793, 8211, 1857],
        [6689, 8108, 4518, ..., 2227, 7883,  440],
        [ 827, 3470, 2522, ...,  333, 6869, 5720]])]

In [71]:
AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand = getAll_RandomHitRate(newHOLDOUT_LIST,All_RandPred,'Updated_UserID','Updated_ItemID')

Number of hits:  0
Total Num of users:  178
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  173
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  207
Recommendation HitRate:  0.0
Number of hits:  1
Total Num of users:  165
Recommendation HitRate:  0.006060606060606061
Number of hits:  0
Total Num of users:  115
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  72
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.0007575757575757576


### Most Pop Rec

In [ ]:
# MostPOPPred_ = getMOSTPOP_Pred(new_PSIDFlist[0],'Updated_UserID','Updated_ItemID',Nusers=10,N=10)
# MostPOPPred_

In [73]:
All_MostPOPRED_List =  getAll_MOSTPOP_Pred(new_PSIDFlist,newHOLDOUT_LIST,'Updated_UserID','Updated_ItemID',N=10)
All_MostPOPRED_List[:2]

[array([[ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        ...,
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106]]),
 array([[110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        ...,
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67]])]

In [74]:
AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand  = getAll_MostPOPHitRate(newHOLDOUT_LIST,All_MostPOPRED_List,'Updated_UserID','Updated_ItemID')

Number of hits:  22
Total Num of users:  178
Recommendation HitRate:  0.12359550561797752
Number of hits:  23
Total Num of users:  173
Recommendation HitRate:  0.1329479768786127
Number of hits:  57
Total Num of users:  207
Recommendation HitRate:  0.2753623188405797
Number of hits:  32
Total Num of users:  165
Recommendation HitRate:  0.19393939393939394
Number of hits:  18
Total Num of users:  115
Recommendation HitRate:  0.1565217391304348
Number of hits:  1
Total Num of users:  72
Recommendation HitRate:  0.013888888888888888
Number of hits:  4
Total Num of users:  44
Recommendation HitRate:  0.09090909090909091
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.12339561427562232


In [75]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8
